In [68]:
# import models
import mplfinance as mpf
from config import api_key
import requests
import pandas as pd
import time
import datetime
from datetime import time
import matplotlib.pyplot as plt
import cufflinks as cf
import os


In [70]:
stocks = ["AAPL", "NVDA", "AMZN", "TSLA", "MSFT", "GOOG", "META", "GOOGL", "PEP", "COST", "AVGO", "ADBE", "CSCO", "CMCSA", "TMUS", "INTC", "TXN", "AMD", "QCOM", "AMGN", "HON", "INTU", "FISV", "MRNA", "PANW", "KDP", "KLAC", "MNST", "SNPS", "AEP"]

In [71]:

periodtype = "year"
frequency = "1"
frequencytype = "monthly"
period = "20"
needExtendedHoursData = "False"

In [72]:
for symbol in stocks:
    #url for td
    url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)

    #parameters
    key = {'apikey':api_key,
             'periodType':periodtype,
              'frequencyType': frequencytype,
              'frequency': frequency,
              'period': period,
              'needExtendedHoursData': needExtendedHoursData,
             }
    #requesting data
    content = requests.get(url = url, params=key)

    data = content.json()

    #create arrays for each category
    open_tick = []
    high_tick = []
    low_tick = []
    close_tick = []
    volume_tick = []
    datetime_tick = []
    #iterrates through data to append each array with it's value
    for i in range(len(data["candles"])):
        open_tick.append(data["candles"][i]["open"])
        high_tick.append(data["candles"][i]["high"])
        low_tick.append(data["candles"][i]["low"])
        close_tick.append(data["candles"][i]["close"])
        volume_tick.append(data["candles"][i]["volume"])
        datetime_tick.append(data["candles"][i]["datetime"])

    #converting epoch to datetime       
    file = []
    for i in range(len(datetime_tick)):
        # divide epoch by 1000 because it is in milliseconds
        test = datetime_tick[i]/1000
        file.append(datetime.datetime.fromtimestamp(test))

    #convert to dataframe
    theta_df = pd.DataFrame({
                "Open":open_tick,
                "Close":close_tick,
                "High":high_tick,
                "Low":low_tick,
                "Volume":volume_tick,
                "Datetime": file
        })
    # set index to Datetime
    theta_df = theta_df.set_index("Datetime")

    # Calculate % gain for the day 
    theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

    # for the stock ticker if a folder does not exist already, one will be created
    mypath = f"{symbol}"
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

    # uploads csv to that folder specifying the frequency and type to identify the csv
    upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
    theta_df.to_csv(upload_file, index = True)